<a href="https://colab.research.google.com/github/natedejong/machine-learning/blob/main/ludwig_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
!pip install ludwig[viz,test]

     |████████████████████████████████| 388 kB 23.8 MB/s 
     |████████████████████████████████| 22.3 MB 3.7 MB/s 
     |████████████████████████████████| 495.0 MB 18 kB/s 
     |████████████████████████████████| 1.1 MB 58.1 MB/s 
     |████████████████████████████████| 134 kB 49.8 MB/s 
     |████████████████████████████████| 47 kB 3.9 MB/s 
     |████████████████████████████████| 96 kB 5.1 MB/s 
     |████████████████████████████████| 29.2 MB 1.5 MB/s 
     |████████████████████████████████| 53 kB 2.1 MB/s 
     |████████████████████████████████| 53 kB 1.7 MB/s 
     |████████████████████████████████| 10.9 MB 57.8 MB/s 
     |████████████████████████████████| 6.0 MB 49.1 MB/s 
     |████████████████████████████████| 1.8 MB 52.7 MB/s 
     |████████████████████████████████| 1.2 MB 51.1 MB/s 
     |████████████████████████████████| 862 kB 28.8 MB/s 
     |████████████████████████████████| 3.4 MB 39.2 MB/s 
     |████████████████████████████████| 284 kB 54.5 MB/s 
     |███████████████

# EDA

In [25]:
# upload the CSV file from BTS to session files
# https://data.bts.gov/Research-and-Statistics/Port-and-Commodity-TransBorder-January-2006-to-Lat/ku5b-t97n
raw_file = "/content/Port_and_Commodity_TransBorder_-_January_2006_to_Latest_Available_Month.csv"
raw_df = pd.read_csv(raw_file)
print(raw_df.head())
print(raw_df.shape)

   Trade Type Port Code  Commodity Code  ...  Container Code  Month  Year
0           1       101              84  ...               X      1  2006
1           1       102              27  ...               X      1  2006
2           1       102              48  ...               X      1  2006
3           1       103               7  ...               X      1  2006
4           1       103              39  ...               X      1  2006

[5 rows x 12 columns]
(3225418, 12)


In [13]:
# Shipping Weight: about 1/3 of values are 0
# Value in US Dollars: no $0s or blanks. the one metric Uncle Sam ensures is complete :)
# Freight Charges: 1/4 of values are 0 (we have to drop these rows since it's the target var)

print(raw_df.nunique())

Trade Type                            2
Port Code                           305
Commodity Code                       98
Mode of Transportation                8
Country                               2
Value in US Dollars             1309415
Shipping Weight in Kilograms     531745
Freight Charges                  292833
Domestic - Foreign Code               2
Container Code                        3
Month                                12
Year                                 16
dtype: int64


In [28]:
raw_df['Value in US Dollars'].value_counts()

3000         2304
5000         1885
4000         1282
6000         1251
10000        1211
             ... 
6590788         1
12082133        1
18303158        1
716908          1
205909444       1
Name: Value in US Dollars, Length: 1309415, dtype: int64

In [23]:
# remove rows with no freight charge history
df = raw_df.drop(raw_df.index[raw_df['Freight Charges']==0])

# Ludwig prediction

In [31]:
!ludwig experiment \
  --dataset 'Port_and_Commodity_TransBorder_-_January_2006_to_Latest_Available_Month.csv' \
  --config model_def_transborder.yaml

/usr/local/lib/python3.7/dist-packages/horovod/common/util.py:258: UserWarning: Framework tensorflow installed with version 2.8.0 but found version 2.7.1.
             This can result in unexpected behavior including runtime errors.
             Reinstall Horovod using `pip install --no-cache-dir` to build with the new version.
  warnings.warn(get_version_mismatch_message(name, version, installed_version))
███████████████████████
█ █ █ █  ▜█ █ █ █ █   █
█ █ █ █ █ █ █ █ █ █ ███
█ █   █ █ █ █ █ █ █ ▌ █
█ █████ █ █ █ █ █ █ █ █
█     █  ▟█     █ █   █
███████████████████████
ludwig v0.4.1 - Experiment

2022-02-25 05:13:25.065608: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Experiment name: experiment
Model name: run
Output directory: results/experiment_run


ludwig_version: '0.4.1'
command: ('/usr/local/bin/ludwig experiment --dataset '
 'Port_and_Commodity_TransBorder_-_January_2006_to_Latest_Availab

In [40]:
!ludwig visualize --visualization learning_curves \
 --training_statistics results/experiment_run/training_statistics.json \
 --output_directory /content/drive/MyDrive/Data\ Science/results/graphs

/usr/local/lib/python3.7/dist-packages/horovod/common/util.py:258: UserWarning: Framework tensorflow installed with version 2.8.0 but found version 2.7.1.
             This can result in unexpected behavior including runtime errors.
             Reinstall Horovod using `pip install --no-cache-dir` to build with the new version.
  warnings.warn(get_version_mismatch_message(name, version, installed_version))


In [34]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [38]:
!cp /content/model_def_transborder.yaml /content/drive/MyDrive/Data\ Science